# Задание 1 Загрузка данных

Изучить представленный набор данных на основе описания его столбцов в файле “horse_data.names” , загрузить его и оставить 8 столбцов для дальнейшего изучения: surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome.

In [40]:
import pandas as pd

# Согласно замечанию в ревью:
# При чтении файла в read_csv() библиотеки Pandas мы можем указать, какие столбцы хотим читать, чтобы не загружать все данные, например это можно сделать с помощью параметра usecols, который в качестве значения принимает список индексов тех столбцов, которые нужно читать.
# Имена заголовкам столбцов можно задать с помощью параметра names.
# Учесть пропуски при чтении файла можно с помощью параметра na_values.

df = pd.read_csv('horse_data.csv',
                header=None,
                # индексы
                usecols=[0, 1, 3, 4, 5, 6, 10, 22],
                # имена
                names=['surgery?',
                       'Age',
                       'rectal_temperature',
                       'pulse',
                       'respiratory_rate',
                       'temperature_of_extremities',
                       'pain',
                       'outcome'],
                # типы
                dtype={0: 'float',
                       1: 'float',
                       3: 'float',
                       4: 'float',
                       5: 'float',
                       6: 'float',
                       10: 'float',
                       22: 'float'},
                # пропуски
                na_values='?')

df.head()

,surgery?,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
0,2.0,1.0,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1.0,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,1.0,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,9.0,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,1.0,37.3,104.0,35.0,NaN,NaN,2.0


# Задание 2. Первичное изучение данных
Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

In [13]:
# Оценка структуры данных

# Согласно замечанию в ревью:
# Использование функции print() при работе с Pandas может быть неэффективным, поскольку она выводит на экран текстовое представление объекта Pandas, которое может быть не очень удобочитаемым для больших и сложных датафреймов.
# Кроме того, форматирование вывода с помощью функции print() может отличаться от того, как Pandas отображает таблицы.

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery?                    299 non-null    float64
 1   Age                         300 non-null    float64
 2   rectal_temperature          240 non-null    float64
 3   pulse                       276 non-null    float64
 4   respiratory_rate            242 non-null    float64
 5   temperature_of_extremities  244 non-null    float64
 6   pain                        245 non-null    float64
 7   outcome                     299 non-null    float64
dtypes: float64(8)
memory usage: 18.9 KB


In [14]:
df.describe()

,surgery?,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
count,299.000000,300.000000,240.000000,276.000000,242.000000,244.000000,245.00000,299.000000
mean,1.397993,1.640000,38.167917,71.913043,30.417355,2.348361,2.95102,1.551839
std,0.490305,2.173972,0.732289,28.630557,17.642231,1.045054,1.30794,0.737187
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.00000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,18.500000,1.000000,2.00000,1.000000
50%,1.000000,1.000000,38.200000,64.000000,24.500000,3.000000,3.00000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.00000,2.000000
max,2.000000,9.000000,40.800000,184.000000,96.000000,4.000000,5.00000,3.000000


In [15]:
df.head()

,surgery?,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
0,2.0,1.0,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1.0,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,1.0,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,9.0,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,1.0,37.3,104.0,35.0,NaN,NaN,2.0


In [16]:
df.tail()

,surgery?,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
295,1.0,1.0,NaN,120.0,70.0,4.0,2.0,3.0
296,2.0,1.0,37.2,72.0,24.0,3.0,4.0,3.0
297,1.0,1.0,37.5,72.0,30.0,4.0,4.0,2.0
298,1.0,1.0,36.5,100.0,24.0,3.0,3.0,1.0
299,1.0,1.0,37.2,40.0,20.0,NaN,NaN,3.0


In [17]:
df.sample()

,surgery?,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
232,1.0,1.0,38.5,30.0,18.0,NaN,NaN,1.0


In [35]:
# Согласно замечанию в ревью:
# в столбце Age есть значения 9, которые лежат вне интервала возможных значений (1, 2)

# словарь с допустимыми значениями для каждого столбца с категориями
valid_values = {
    'surgery?': [1, 2],
    'Age': [1, 2],
    'temperature_of_extremities': [1, 2, 3, 4],
    'pain': [1, 2, 3, 4, 5],
    'outcome': [1, 2, 3]
}

def count_invalid_values():
    # Проверка недопустимых значений и подсчет их количества
    invalid_counts = {}

    for column, valid in valid_values.items():
        # Подсчитываем количество недопустимых значений
        invalid_count = df[~df[column].isin(valid) & df[column].notnull()].shape[0]
        invalid_counts[column] = invalid_count

    # Выводим количество недопустимых значений для каждой колонки
    for column, count in invalid_counts.items():
        print(f"Количество недопустимых значений в колонке '{column}': {count}")

print("до")
count_invalid_values()

# Замена недопустимых значений
for column, valid in valid_values.items():
    if column == 'Age':
        # Заменяем все значения в Age на 2, если они недопустимы
        df[column] = df[column].where(df[column].isin(valid), 2)
    else:
        # Для всех остальных колонок заменяем недопустимые значения на None
        df[column] = df[column].where(df[column].isin(valid), None)

print("\nпосле")
count_invalid_values()


до
Количество недопустимых значений в колонке 'surgery?': 0
Количество недопустимых значений в колонке 'Age': 24
Количество недопустимых значений в колонке 'temperature_of_extremities': 0
Количество недопустимых значений в колонке 'pain': 0
Количество недопустимых значений в колонке 'outcome': 0

после
Количество недопустимых значений в колонке 'surgery?': 0
Количество недопустимых значений в колонке 'Age': 0
Количество недопустимых значений в колонке 'temperature_of_extremities': 0
Количество недопустимых значений в колонке 'pain': 0
Количество недопустимых значений в колонке 'outcome': 0


In [32]:
# Согласно замечанию в ревью:
# Столбцы surgery?, Age, temperature of extremities, pain, outcome категориальные,
# Для них можно найти моды, проанализировать распределение по категориям, проверить уникальные значения категорий на предмет ошибок и соответствия описанию данных. Помогут методы mode(), value_counts(), unique().

# Базовые статистики для числовых столбцов
numeric_cols = ['rectal_temperature', 'pulse', 'respiratory_rate']
numeric_stats = df[numeric_cols].describe(percentiles=[.25, .5, .75])

# Анализ категориальных столбцов
categorical_cols = ['surgery?', 'Age', 'temperature_of_extremities', 'pain', 'outcome']
categorical_analysis = {}
for col in categorical_cols:
    modes = df[col].mode().tolist()
    value_counts = df[col].value_counts(dropna=False)
    unique_values = df[col].unique()
    categorical_analysis[col] = {
        'mode': modes,
        'value_counts': value_counts,
        'unique_values': unique_values
    }

# Поиск выбросов для числовых столбцов с помощью IQR
outliers = {}
for col in numeric_cols:
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outlier_mask = (df[col] < lower_bound) | (df[col] > upper_bound)
    outliers[col] = df[col][outlier_mask].tolist()

# Результаты
print("Базовые статистики для числовых столбцов:\n", numeric_stats)
print("\nАнализ категориальных столбцов:")
for col, data in categorical_analysis.items():
    print(f"\nСтолбец: {col}")
    print(f"Мода: {data['mode']}")
    print(f"Распределение категорий:\n{data['value_counts']}")
    print(f"Уникальные значения: {data['unique_values']}")

print("\nВыбросы в числовых столбцах:")
for col, values in outliers.items():
    print(f"{col}: {values}")

Базовые статистики для числовых столбцов:
        rectal_temperature       pulse  respiratory_rate
count          240.000000  276.000000        242.000000
mean            38.167917   71.913043         30.417355
std              0.732289   28.630557         17.642231
min             35.400000   30.000000          8.000000
25%             37.800000   48.000000         18.500000
50%             38.200000   64.000000         24.500000
75%             38.500000   88.000000         36.000000
max             40.800000  184.000000         96.000000

Анализ категориальных столбцов:

Столбец: surgery?
Мода: [1.0]
Распределение категорий:
surgery?
1.0    180
2.0    119
NaN      1
Name: count, dtype: int64
Уникальные значения: [ 2.  1. nan]

Столбец: Age
Мода: [1.0]
Распределение категорий:
Age
1.0    276
2.0     24
Name: count, dtype: int64
Уникальные значения: [1. 2.]

Столбец: temperature_of_extremities
Мода: [3.0]
Распределение категорий:
temperature_of_extremities
3.0    109
1.0     78
NaN   

In [5]:
# Анализируем выбросы колонки "ректальная температура". Тут хотя бы числа, остальные колонки - категории.
# странно анализировать отсутствие информации на предмет выборосов; игнорируем пустышки
df_cleaned = df.dropna(subset=['rectal_temperature'])
# Средняя температура
print(f"Средняя температура: {df_cleaned['rectal_temperature'].mean()}")
# тык от 35 до 40 градусов (common sense... ходят слухи, что белок в клетках сворачивается при температуре выше 40 градусов)
print(df_cleaned[(df_cleaned['rectal_temperature'] > 35) & (df_cleaned['rectal_temperature'] < 40)]['rectal_temperature'].mean())
print()
# медиана
print(f"Медиана: {df_cleaned['rectal_temperature'].median()}")
print(df_cleaned[(df_cleaned['rectal_temperature'] > 35) & (df_cleaned['rectal_temperature'] < 40)]['rectal_temperature'].median())
print()
# мода
print(f"Мода: {df_cleaned['rectal_temperature'].round().mode()[0]}")
print(df_cleaned[(df_cleaned['rectal_temperature'] > 35) & (df_cleaned['rectal_temperature'] < 40)]['rectal_temperature'].round().mode()[0])
print()

# Выбросы через межквартильный размах
q1 = df_cleaned['rectal_temperature'].quantile(0.25)
q3 = df_cleaned['rectal_temperature'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr)
upper_bound = q3 + (1.5 * iqr)
remove_outliers = df_cleaned['rectal_temperature'].between(lower_bound, upper_bound, inclusive='both')
# что это за выбросы?
df_cleaned[~df_cleaned['rectal_temperature'].between(lower_bound, upper_bound, inclusive='both')]

Средняя температура: 38.16791666666667
38.13093220338982

Медиана: 38.2
38.150000000000006

Мода: 38.0
38.0



,surgery?,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
20,1,1,39.9,72,60,1,5,1
44,1,1,35.4,140,24,3,4,3
54,2,1,40.3,114,36,3,2,3
75,1,9,39.7,100,<NA>,3,2,3
80,1,1,36.4,98,35,3,4,2
91,2,1,40.3,114,36,3,2,2
99,2,1,39.6,108,51,3,2,1
118,1,1,36.5,78,30,1,5,1
141,2,1,36.0,42,30,<NA>,<NA>,2
238,2,1,36.1,88,<NA>,3,3,3


# Задание 3. Работа с пропусками
Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.

In [ ]:
# Согласно замечанию в ревью:
# При заполнении пропусков, нам достаточно руководствоваться простыми правилами статистики,
# Для категориальных данных лучшим способом замены пропусков является мода.
# Для количественных данных, если в непрерывной величине есть выбросы, то стоит выбрать для замены медиану, если в непрерывной величине нет выбросов, то стоит выбрать среднее.

df_simple_filled = df.copy()
print("дo:")
print(df_simple_filled.isnull().sum())

# В рамках 'Задание 2' мы уже выяснили, что у нас есть следующие выбросы:
# rectal_temperature: [39.9, 35.4, 40.3, 39.7, 36.4, 40.3, 39.6, 36.5, 36.0, 36.1, 36.6, 40.8, 40.0, 36.5]
# pulse: [164.0, 150.0, 160.0, 184.0, 150.0]
# respiratory_rate: [84.0, 96.0, 72.0, 80.0, 80.0, 68.0, 96.0, 66.0, 68.0, 90.0, 80.0, 70.0, 88.0, 84.0, 68.0, 90.0, 70.0]

# Поэтому заполняем числовые пропуски медианой
df_simple_filled['rectal_temperature'] = df_simple_filled['rectal_temperature'].fillna(df_simple_filled['rectal_temperature'].median())
df_simple_filled['pulse'] = df_simple_filled['pulse'].fillna(df_simple_filled['pulse'].median())
df_simple_filled['respiratory_rate'] = df_simple_filled['respiratory_rate'].fillna(df_simple_filled['respiratory_rate'].median())

# Заполнение категориальных пропусков модой
df_simple_filled['surgery?'] = df_simple_filled['surgery?'].fillna(df_simple_filled['surgery?'].mode()[0])
df_simple_filled['Age'] = df_simple_filled['Age'].fillna(df_simple_filled['Age'].mode()[0])
df_simple_filled['temperature_of_extremities'] = df_simple_filled['temperature_of_extremities'].fillna(df_simple_filled['temperature_of_extremities'].mode()[0])
df_simple_filled['pain'] = df_simple_filled['pain'].fillna(df_simple_filled['pain'].mode()[0])
df_simple_filled['outcome'] = df_simple_filled['outcome'].fillna(df_simple_filled['outcome'].mode()[0])

# Проверка пропусков после заполнения
print("\nпосле:")
print(df_simple_filled.isnull().sum())

дo:
surgery?                       1
Age                            0
rectal_temperature            60
pulse                         24
respiratory_rate              58
temperature_of_extremities    56
pain                          55
outcome                        1
dtype: int64

после:
surgery?                      0
Age                           0
rectal_temperature            0
pulse                         0
respiratory_rate              0
temperature_of_extremities    0
pain                          0
outcome                       0
dtype: int64


In [ ]:
# Согласно замечанию в ревью:
# Заполнение пропусков на основе других признаков, категорий и группировок может быть точнее, чем просто замена пропущенных значений на модальные, средние или медианные значения по конкретным столбцам
# например, что средняя температура или другие метрики отличаются у особей разных возрастных групп
# Или для тех, кому требуется операция (и она была сделана) и для тех, кому нет.

df_by_category_filled = df_simple_filled.copy()

# возвращаем значения колонок на исходные (с пропусками)
# считаем, что температура зависит от возраста
df_by_category_filled['rectal_temperature'] = df['rectal_temperature'] # количественный
df_by_category_filled['temperature_of_extremities'] = df['temperature_of_extremities'] # категория
# считаем, что исход зависит от уровня боли
df_by_category_filled['pulse'] = df['pulse']
# считаем, что дыхание зависит от того, была ли операция
df_by_category_filled['respiratory_rate'] = df['respiratory_rate']

print("дo:")
print(df_by_category_filled.isnull().sum())

# Количественные признаки
# заполняем температуру в зависимости от возраста
df_by_category_filled['rectal_temperature'].fillna(df_by_category_filled.groupby('Age')['rectal_temperature'].transform('median'))
# заполняем исход в зависимости от уровня боли
df_by_category_filled['pulse'].fillna(df_by_category_filled.groupby('pain')['pulse'].transform('median'))
# заполняем дыхание в зависимости от того, была ли операция
df_by_category_filled['respiratory_rate'].fillna(df_by_category_filled.groupby('surgery?')['respiratory_rate'].transform('median'))

# Категории
# Заполняем температуру экскрементов в зависимости от возраста
df_by_category_filled['temperature_of_extremities'].fillna(df.groupby('Age')['temperature_of_extremities'].transform(lambda x: x.mode()[0]))

# Проверка пропусков после заполнения
print("\nпосле:")
print(df_simple_filled.isnull().sum())

дo:
surgery?                       0
Age                            0
rectal_temperature            60
pulse                         24
respiratory_rate              58
temperature_of_extremities    56
pain                           0
outcome                        0
dtype: int64

после:
surgery?                      0
Age                           0
rectal_temperature            0
pulse                         0
respiratory_rate              0
temperature_of_extremities    0
pain                          0
outcome                       0
dtype: int64
